<a href="https://colab.research.google.com/github/YuezhiMao/Teaching-materials/blob/main/chem713/Variational_method_and_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Practice Session 3: Approximate Methods: Linear Variational Method and Non-Degenerate Perturbation Theory**

In this practice session, we will use Python to demonstrate the application of two important approximate methods in quantum mechanics: (i) the **linear variational method** and (ii) **non-degenerate perturbation theory**.
This practice will involve less tutorial of Python, and we will pay more attention to the physics instead. At the same time, you will be practicing a couple of things you learned in the last two practice:


1.   Definition of mathematical functions
2.   Matrix algebra, in particular solving for eigenvalues of Hermitian matrices
3.   Calculating integrals of simple mathematical functions
4.   Plotting mathematical functions using Matplotlib

While the usage of a few essential functions will still be provided in this Notebook, please either refer to the Python Notebooks for the previous two Practice sessions or look up the documentation online (a very useful skill you should really acquire if you want to learn programming by yourself someday). Please also feel free to get help from the instructor for any questions in this practice.

Let's start with importing all the modules we need today, by running the code block below:

In [ ]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt

## 3.1 Setup of the problem

Let's now set up the problem we are going to solve today: it is still a particle-in-box (PIB) problem, but now there is a **Gaussian potential** centered at the center of the box. The potential energy function $V(x)$ now looks like:
\begin{equation*}
V(x) =
\begin{cases}
V_0\exp\left[-\alpha (x - L/2)^2 \right] & 0 \leq x \leq L \\
\infty & x > L \ \mathrm{or} \ x < 0 \\
\end{cases}
\end{equation*}
where $V_0$ represents the magnitude of this potential and $\alpha$ is related to its width. The wave funciton will still be zero everywhere out of the box, while the Schrödinger equation inside the box now has form cannot be solved exactly. In the following, we will utilize the solutions to the problem that we know how to solve, i.e., PIB with no potential inside (we will refer to it as **original PIB**), to approximately solve the new Gaussian potential problem using (i) **linear variational method** and (ii) **non-degenerate perturbation theory**.

Recall that for a 1-D box from 0 to $L$, the $n$-th PIB state (denoted as $\phi_n(x)$ here) is
\begin{equation*}
\phi_n(x) = \sqrt{\frac{2}{L}} \sin\left(\frac{n\pi x}{L}\right)
\end{equation*}
And its energy eigenvalue $E_n$ is
\begin{equation*}
E_n = \frac{n^2h^2}{8mL^2}
\end{equation*}
For convenientce, we will use **atomic units** (a.u.) in this practice (otherwise we will have to carry the Planck constant and the electron mass everywhere). Under atomic units, we have $\hbar = 1$, $m_e = 1$ (electron mass), and $e = 1$ (unit charge). The length unit then becomes $a_0$ (the Bohr radius $\approx 53$ pm) and the unit for energy becomes Ha (hartree, 1 Ha $\approx$ 27.2 eV). Under atomic units, we can write the PIB energy as
\begin{equation*}
E_n = \frac{n^2 \pi^2 \hbar^2}{2mL^2} = \frac{n^2 \pi^2}{2mL^2} (\mathrm{Ha})
\end{equation*}

**Practice 3.1 (1 point)** Complete the following two functions with the given parameters. For the PIB energy, return it in the atomic unit (Ha). The constant $\pi$ can be obtained from NumPy using `np.pi` and so is the $\sin()$ function: `np.sin(x)`

In [ ]:
def pib_wfn(x, n, L):
  # Complete the line below
  return

In [ ]:
def pib_eng(n, m, L):
  # Note that in atomic unit, hbar = 1
  # Complete the line below
  return

The Gaussian potential centered at $x = L/2$ is defined by the function below (simply run this code block). One new usage that you should check out here is that one can set *default value* for parameters passed to a function when we define the function, just like what is done here for the $\alpha$ parameter.
When you call this function afterwards, you can simply write `V_gauss(x, L, V_0)` if you are not asked to modify the value of $\alpha$. The default value 1.0 will be used.

In [ ]:
def V_gauss(x, L, V_0, alpha=1.0):
  return V_0 * np.exp(-alpha*(x - L/2)**2)

To further simplify the discussion, we can further set two parameters ($L$ and $m$) that won't be changed throughtout this practice (run the code block below). Here we set $L = 10a_0$ and the mass equal to electron mass (i.e., considering the particle to be an electron). Under atomic units, we simply have $m = 1$.

In [ ]:
L = 10  # box size: 10a_0
m = 1   # electron mass in atomic unit

Below is the code to plot the Gaussian potential with $V_0 = 0.02$ using Matplotlib, which can be used as a template for plotting functions later in this exercise. Run the code block below and check out how our $V(x)$ looks like in the box.

In [ ]:
x = np.linspace(0, 10, 101)
plt.plot(x, V_gauss(x, L, V_0 = 0.02), label='$V(x)$')
plt.xlabel('$x$ ($a_0$)', fontsize=14)
plt.ylabel('$V_{\mathrm{gauss}}$ (Ha)', fontsize=14)
plt.xlim(0, 10)
plt.legend(fontsize=12)
plt.show()

##3.2 Linear Variational Method

As introduced in our lecture, in **linear variational method** we choose a set of basis functions and then express the trial function as a *linear expansion* in this basis:
\begin{equation*}
|\tilde{\Psi}⟩ = \sum_{i=1}^{N} |\phi_i⟩ c_i
\end{equation*}
where $c_i$'s are expansion coefficients which are to be optimized, and $N$ is the number of basis functions. The variational optimization of the $c_i$'s is equivalent to solving a matrix equation:
\begin{equation*}
\mathbf{H}\mathbf{c} = E\mathbf{Sc}
\end{equation*}
where $\mathbf{H}$ and $\mathbf{S}$ are the overlap matrices:
\begin{align*}
H_{ij} &= ⟨\phi_i | \hat{H} | \phi_j ⟩ = \int dx \phi_i(x) \hat{H} \phi_j(x) \\
S_{ij} &= ⟨\phi_i | \phi_j ⟩ = \int dx \phi_i(x) \phi_j(x)
\end{align*}
and the $i$-th column of $\mathbf{c}$ corresponds to the linear combination coefficients for the $i$-th state (the first one corresponding to the lowest energy eigenvalue is the ground state). Note that here we restrict ourselves to *real* basis functions and *real* expansion coefficients so in the calculation of integrals we can leave out the asterisks for complex conjugate.

After reviewing the basics, let's now go back to our specific problem here. A very natural choice is to use the eigenfunctions of the original PIB (defined by function `pib_wfn()` above) as the basis and we truncate the expansion at a given $N$, which will be denoted as `NBas` in our program. A nice feature of these functions is that they are **orthonormal functions** (i.e., $\langle \phi_i | \phi_j \rangle = \delta_{ij}$) so that the matrix equation reduces to a standard Hermitian matrix eigenvalue problem:
\begin{equation*}
\mathbf{Hc} = E\mathbf{c}
\end{equation*}

Next we need to construct the Hamiltonian matrix. To work out the expression for the matrix elements, we can first rewrite the Hamiltonian as
\begin{equation*}
\hat{H} = \hat{H}_0 + \hat{V}
\end{equation*}
where $\hat{H}_0$ is Hamiltonian of the original PIB ($\hat{H}_0 = \hat{p}^2/2m$), which satisfies $\hat{H}_0 |\phi_i⟩ = E_i |\phi_i⟩$.







**Practice 3.2 (2 pts)** If we denote
\begin{equation*}
V_{ij} = \langle \phi_i | \hat{V} | \phi_j \rangle = \int_0^{L} dx \phi_i(x) V(x) \phi_j(x)
\end{equation*}
Derive the form of $H_{ii}$ (diagonal elements of the Hamiltonian) and $H_{ij} (j \neq i)$ (off-diagonal elements) expressed **in terms of the PIB energies and matrix elements of $\mathbf{V}$** (**0.5 pt**, write down the equations in the text box below).

Then, based on your derivation, **complete the following function** which builds the Hamiltonian matrix element by element and then calculate the eigenvalues. **(1.5 pts)**

**A few useful tips/things to learn:**

*   If you are unsure about the derivation of $H_{ii}$ and $H_{ij}$, ask the instructor to check the results for you since it is **essential to get them right** for the rest of this practice
*   Make use of the functions that have been defined: `pib_eng(i, m, L)` for $E_i$'s, `pib_wfn(x, i, L)` for $\phi_i(x)$'s, and `V_gauss(x, L, V_0)` for $V(x)$; $m$ and $L$ are constants that have been defined so you can simply substitute them in.
*   Here we have used an `if` statement to separate the two cases (diagonal and off-diagonal elements). We also used a double `for` loop to go through the row (loop of $i$) and column (loop of $j$) indices. Here we have $j$ starts from $i$ so we only calculated the **upper triangular** part of the matrix (diagonal elements plus all elements above the diagonal); the rest can be obtained based on symmetry since $\hat{H}$ is a Hermitian operator and $\phi_i(x)$'s are real.
* **One tricky thing** that must be noted is that with `for i in range(1, NBas+1)`, the index $i$ actually goes *from 1 to NBas*. This is natural for the `pib_eng` and `pib_wfn` functions where the energy level $n$ starts from 1, but then we will have to write `H[i-1, j-1]` for $H_{ij}$ since the index for elements in NumPy arrays/matrices starts from 0.
*   Recall the usage of the `quad()` function under `scipy.integrate`: we can calculate $I = \int_{a}^{b} dx f(x)$ by
```
I = integrate.quad(lambda x: f(x), a, b)[0]
```
Don't forget to add `[0]` at the end since otherwise the returned results consist of two numbers, with the 2nd being the numerical error of this integral evaluation.
*   At the end, we use `np.linalg.eigh(H)` to calculate the eigenvalue and eigenvectors: the returned `E` contains all the eigenvalues (in ascending order) and the $i$-th column of `c` contains the linear expansion coefficients for the $i$-th eigenstate.

**Part 1 of Practice 3.2**

The expression for Hamiltonian matrix elements: **(double-click to edit)**


*   The diagonal element:
\begin{equation*}
H_{ii} = ?
\end{equation*}
*   The off-diagonal element:
\begin{equation*}
H_{ij} = ?
\end{equation*}


**Part 2: Complete the function below**

In [ ]:
def solve_secular_eqn(NBas, V_0):
  #Start from a NBas-by-NBas filled by zeros
  H = np.zeros((NBas, NBas))
  #Loop over the matrix elements: doing the upper triangular part only
  for i in range(1, NBas+1):
    for j in range(i, NBas+1):
      if i == j:
        # Your work here: translate your expression of H_ii into Python code
        H[i-1, i-1] =
      else:
        # Your work here: translate your expression of H_ij into Python code
        H[i-1, j-1] =
        # Take advantage of the property of Hermitian operator
        H[j-1, i-1] = H[i-1, j-1]

  #Solve the Hermitian matrix eigenvalue problem
  E, c = np.linalg.eigh(H)
  return E, c

**Practice 3.3 (1.5 pts)** With $V_0$ set at 0.02 Ha, run the code block below with `NBas` equal to 5, 10, 15, 20, and 50. Check the obtained energies of the first and second eigenstates and answer the following questions:


1.   For the 1-D array `E` returned by the `solve_secular_eqn` function, does it always have the same dimension? What should be the dimension of this array? Feel free to check this by printing things out in the code if you are unsure. **(0.5 pt)**
2.   Does the energy go down monotonically with the increasing number of basis functions? Does this indicate that using more basis functions in linear variational method gives us more accurate results? **(0.5 pt)**
3.   If we require an error below 10$^{-10}$ a.u, how many basis functions do we need to converge the results for $E_1$, $E_2$, and $E_3$? (Here we print out 10 digits so you just need to check when the results stop to change when more basis functions are added.) **(0.5 pt)**



In [ ]:
V_0 = 0.02
print ("Checking the effect of number of basis functions with V_0 = %.2f" %V_0)
# Using a for loop, we can conveniently scan over a bunch of different values
NBas_values = [5, 10, 15, 20, 50]
for NBas in NBas_values:
  print ("Number of basis functions: %d" %NBas)
  E, c = solve_secular_eqn(NBas, V_0)
  #Again, note that the index for numpy array elements starts from 0
  print ("E_1 = %.10f, E_2 = %.10f, E_3 = %.10f"
         %(E[0], E[1], E[2]))

**Answer the 3 questions in Practice 3.3 here (double-click to edit):**


1.   
2.   
3.   



Now let's switch our focus to the **eigenstates** we obtained by solving the secular equantion (diagonalizing $\mathbf{H}$). Here we denote the $n$-th eigenstate as $|\psi_n⟩$, and recall that
\begin{equation*}
|\psi_n⟩ = \sum_{i=1}^{N_{bas}} |\phi_i⟩ c_{in}  
\end{equation*}
where $|\phi_i\rangle$'s are the basis functions which are simply the original PIB states. We can also rewrite this in terms of wave functions:
\begin{equation*}
\psi_n(x) = \sum_{i=1}^{N_{bas}} \phi_i(x) c_{in}  
\end{equation*}

**Practice 3.4 (0.5 pt)** Complete the function below to produce $\psi_n(x)$. The skeleton of the function has been provided, and what you need to do is to complete the sum (the one line within the `for` loop).

**Hint:** Recall that to obtain the element at its $i$-th row and $j$-th column ($i$ and $j$ both start from 1) in a NumPy matrix $\mathbf{A}$, you simply need \\
`A[i-1, j-1]`

**Note:** This is also an important function which is essential to the plotting tasks below. If you are unsure about what you have put in there, ask the instructor to check it for you.





In [ ]:
def variational_solution(x, c, n):
  # NBas is equal to the length of each eigenvector
  NBas = c.shape[0]
  # Set the initial value for psi
  psi = 0.0
  for i in range(1, NBas+1):
    # Your work: complete the sum here
    # "+=" means adding the quantity on the right-hand side on top of the one on the left
    psi +=
  return psi

To check if you have defined the above function correclty and to see how it can be called, check out and also run the code block below. Here we are just verifying the *orthonormal* condition between the linear variational solutions, i.e., approximate solutions to the Gaussian potential PIB problem. For two eigenstates $|\psi_m⟩$ and $|\psi_n⟩$ obtained from solving $\mathbf{Hc} = E\mathbf{c}$, they should satisfy
\begin{equation*}
\mathcal{S}_{mn} = \delta_{mn}
\end{equation*}

In [ ]:
# To make things not mixed up, we rerun this here with 50 basis functions and V_0 = 0.02
NBas = 50
V_0 = 0.02
E, c = solve_secular_eqn(NBas, V_0)

S_11 = integrate.quad(lambda x: variational_solution(x, c, 1)**2, 0, L)[0]
print ("S_11 = %.6f" %S_11)
S_12 = integrate.quad(lambda x: variational_solution(x, c, 1)*variational_solution(x, c, 2),
                      0, L)[0]
S_23 = integrate.quad(lambda x: variational_solution(x, c, 2)*variational_solution(x, c, 3),
                      0, L)[0]
print ("S_12 = %.6f; S_23 = %.6f" %(S_12, S_23))

**Practice 3.5 (1 pt)** We are now going to *visualize* the first 3 eigenstates $|\psi_1⟩$, $|\psi_2⟩$, and $|\psi_3⟩$ using Matplotlib. A template function has been provided in the code block below, with which $\psi_1(x)$ will be plotted. Now add the following to the plot function and check out the plot:


1.   Plot the ground state of the original PIB problem $\phi_1(x)$ also in **red** color but with `linestyle='dashed'` (**Hint:** you should be using the `pib_wfn` function for this task).
2.   With the same settings, plot $\psi_2(x)$ and $\phi_2(x)$ in blue, and $\psi_3(x)$ and $\phi_3(x)$ in green.



In [ ]:
def plot_variational_solution(c):
  x = np.linspace(0, 10, 101)
  # Plot psi_1(x)
  plt.plot(x, variational_solution(x, c, 1), label='$\psi_1(x)$', color='red')
  # Your work starts here: first, plot phi_1(x)

  # Your work: plot psi_2(x) and phi_2(x)


  # Your work: plot psi_3(x) and phi_3(x)



  plt.xlabel('$x$ (a.u.)', fontsize=14)
  plt.ylabel('$V_{\mathrm{gauss}}$ (a.u.)', fontsize=14)
  plt.axhline(y = 0, linewidth=0.5, linestyle='dashdot', color='black')
  plt.xlim(0, 10)
  plt.ylim(-0.6, 0.6)
  plt.legend(fontsize=12)
  plt.show()

#Execute the plotting function defined above
plot_variational_solution(c)

You will notice that the PIB wave functions under the Gaussian potential with $V_0 = 0.02$ Ha are not very different from the original PIB wave functions. This indicates that the coupling between the original PIB states due to the Gaussian potential is weak.

Let's now solve the same linear variational problem but with the potential 5 times stronger ($V_0 = 0.1$ Ha). To make sure things converge properly we use 50 basis functions. Run the **2nd code block below** *without* any modifications.

The first thing you might notice is that $\psi_2(x)$ looks completely different from $\phi_2(x)$, and it seems their phases differ by half a period. This is caused by the arbitrariness of the sign of each of the eigenvectors in $\mathbf{c}$ produced by the `np.linalg.eigh()` function, since $-|\psi_i⟩$ and $|\psi_i⟩$ are mathematically equivalent serving as $\mathbf{H}$'s eigenvectors.

To remedy this, we can compute the overlap between $\phi_i(x)$ and $\psi_i(x)$: if it is negative, then we should flip the sign of the $i$-th column. This procedure can be achieved by the `adjust_phase()` function. Look through and run the code block below, and then add it to the **2nd code block below** right before calling the plotting function. Rerun that code block, and you will see the phases of $\psi_2(x)$ and $\phi_2(x)$ are now matching.

In [ ]:
def adjust_phase(c):
  # Number of columns in c should correspond to the number of states
  NStates = c.shape[1]
  for i in range(1, NStates+1):
    # The overlap between psi_i(x) and phi_i(x)
    overlap = integrate.quad(lambda x: pib_wfn(x, i, L)*variational_solution(x, c, i),
                             0, L)[0]
    if overlap < 0:
      # c[:, i-1] represents the i-th column of c
      c[:, i-1] *= -1

In [ ]:
NBas = 50
V_0 = 0.1
E, c = solve_secular_eqn(NBas, V_0)
print ("E_1 = %.10f, E_2 = %.10f, E_3 = %.10f"
         %(E[0], E[1], E[2]))

# add the adjust_phase function here

plot_variational_solution(c)

**Practice 3.6 (1.5 pt)** Answer the following questions based on the plot you just obtained:


1.   Among the first three states, which one changes the most due to the addition of the Gaussian potential based on the visual comparison of $\psi_i(x)$ and $\phi_i(x)$? Which one is least affected? **(0.5 pt)**
2.   **(1 bonus point)** Explain the physical reason underlying this observation
3.   Copy the last code block above and execute it with another **5$\times$ increase in the strength of the potential** ($V_0 = 0.5$ Ha). Do you find that the patterns you just observed for $V_0 = 0.1$ Ha become more pronounced here?
Then, to conclude this practice about linear variational method, **complete the table below** with the energies for the ground ($E_1$) and first excited state ($E_2$) evaluated using 50 basis functions **(0.5 pt)** (if you do not yet have the energy for the $V_0 = 0$ case, you can simply get it using the `pib_eng()` function). Describe how $E_1$ and $E_2$ vary with increasing strength of the potential **(0.5 pt)**

**Answers to the questions (double-click to edit)**


1.   
2.   
3.    



---
**Ground and 1st excited-state energies for PIB with Gaussian potential evaluated using 50 basis functions** \\
(10 digits after decimal point required)

| $V_0$ (Ha)  | $E_1$ (Ha)            | $E_2$ (Ha)            |
| ----------- | -----------           | -----------           |
| 0           |                       |                       |
| 0.02        |                       |                       |
| 0.1         |                       |                       |
| 0.5         |                       |                       |

In [ ]:
#Copy the last code block and change the value of V_0 as required
NBas = 50
V_0 = 0.5
E, c = solve_secular_eqn(NBas, V_0)
print ("E_1 = %.10f, E_2 = %.10f, E_3 = %.10f"
         %(E[0], E[1], E[2]))

adjust_phase(c)
plot_variational_solution(c)

##3.3 Non-Degenerate Perturbation Theory

Now we will see how we can realize another important approximate method, **non-degenerate perturbation theory** using Python programming to treat the same problem, i.e., **PIB with Gaussian potential**. Recall that we already have a natural partition of the total Hamiltonian:
\begin{equation*}
\hat{H} = \hat{H}_0 + \hat{V}
\end{equation*}
We can simply set the unperturbed Hamiltonian $\hat{H}^{(0)} \equiv \hat{H}_0$, and the first-order Hamiltonian (the "perturbation") $\hat{H}^{(1)} = \hat{V}$.

In our lecture, we have derived that with the energy corrected up to the 2nd order, the energy of state $n$ can be evaluated as
\begin{equation*}
E_n = E_n^{(0)} + E_n^{(1)} + E_n^{(2)}
\end{equation*}
where the 0th-order energy $E_n^{(0)}$ is simply the energy of the original PIB state $n$ (can be obtained using `pib_eng()`). Then the first-order correction to the energy is the **expectation value** of $\hat{H}^{(1)}$ in 0th-order state $n$. In our case, the 0th-order states are just the original PIB eigenstates (can be obtained from the `pib_wfn()` function):
\begin{equation*}
E_n^{(1)} = ⟨\psi_n^{(0)} | \hat{H}^{(1)} | \psi_n^{(0)} \rangle = ⟨\phi_n | \hat{V} | \phi_n\rangle = \int_0^{L} dx \ \phi_n(x) V(x) \phi_n(x)
\end{equation*}
Then for the 2nd-order energy correction $E_n^{(2)}$, we should have
\begin{equation*}
  E_n^{(2)} = \sum_{i\neq n}^{\infty} \frac{\left| ⟨\psi_n^{(0)} | \hat{H}^{(1)} | \psi_i^{(0)}⟩ \right|^2}{E_n^{(0)} - E_i^{(0)}} = \sum_{i\neq n}^{\infty} \frac{\left| ⟨\phi_n | \hat{V} | \phi_i⟩ \right|^2}{E_n^{(0)} - E_i^{(0)}}
\end{equation*}
where the matrix element on the numerator ($V_{ni}$) can evaluated similarly as an integral (which we have done in the linear variational method practice).

Note that In our program, we cannot have infinite sums, and it is totally unnecessary because states that are high up will have no contribution to $E_{n}^{(2)}$ as gated by the denominator. So here we introduce a parameter `Nstate`, which truncates the sum at a certain number.

**Practice 3.7 (2.5 pts)** Complete the function below that evaluate the perturbation correciton to the energy of the $n$-th PIB state. It will return the total energy of state $n$ corrected up to both the 1st ($E_n^{(0)} + E_n^{(1)}$) and 2nd order ($E_n^{(0)} + E_n^{(1)} + E_n^{(2)}$). Instructions are provided as **comments** in the code block. **(1.5 pts)**

As usual, if you are unsure about your code, you can ask the instructor to check it for you :)

In [ ]:
def nondeg_PT(n, Nstate, V_0):
  # 0th-order En should simply be the energy of original PIB state n
  # Complete the line below
  En_0th =
  print ("Original E%d: %.10f" %(n, En_0th))
  # Complete the line below to evaluate the 1st-order correction to En
  En_1st =

  # The first quantity to be reported
  En_1st_corrected = En_0th + En_1st

  # Initialize En_2nd
  En_2nd = 0.0
  # Calculate the sum using a for loop
  for i in range(1, Nstate+1):
    if i != n: # this is required; see the equation above
      # Complete the line below to evaluate the matrix V_ni first
      V_ni =
      # Complete the two lines below to obtain the 2nd-order energy correction
      Ei_0th =
      En_2nd +=

  # The second quantity to be reported
  En_2nd_corrected = En_0th + En_1st + En_2nd

  return En_1st_corrected, En_2nd_corrected

We are now going to calculate the 1st- and 2nd-order corrected energies for the first two states with $V_0 =~$0.02, 0.1, and 0.5. For the infinite sum involved in the evaulation of $E_n^{(2)}$, we truncate at $N = 100$, which should be more than enough (you are encouraged to check out how the results with `Nstate = 20` looks like). **Complete the table below** for $E_1$ and compare with the results of linear variational method (for convenience, you may want to copy the table you completed above and paste it here). Then address the following questions:


1.   If we take the $E_1$ calculating using **linear variational method** with `NBas = 50` as the reference, what are the relative errors (in %) of the 1st- and 2nd-order corrected $E_1$ at $V_0 = 0.02$? **(0.5 pt)**
2.   Focusing on the accuracy of the energies corrected up to the 2nd-order, does the performance degrade with increasing $V_0$? Can you rationalize the behavior you observed? (**Hint:** revisit the prerequisites for the successful application of perturbation theory I mentioned in lecture) **(0.5 pt)**



**Answers to the questions (double-click to edit)**


1.   
2.   
    



---
**Ground state energy for PIB with Gaussian potential corrected up to the 1st and 2nd-order based on non-degenerate perturbation theory (values returned by the nondeg_PT() function)** \\
(10 digits after decimal point required)

| $V_0$ (Ha)  | $E_1$(1st) (Ha)  | $E_1$(2nd) (Ha)       |
| ----------- | -----------      | -----------           |
| 0.02        |                  |                       |
| 0.1         |                  |                       |
| 0.5         |                  |                       |

In [ ]:
Nstate = 100
# Check the same values as in linear variational method
V0_values = [0.02, 0.1, 0.5]

for V_0 in V0_values:
  print("V_0 = %.2f Ha" %V_0)
  #n = 1
  E1_1st_corrected, E1_2nd_corrected = nondeg_PT(1, Nstate, V_0)
  print ("E1 corrected to the 1st order: %.10f" %E1_1st_corrected)
  print ("E1 corrected to the 2nd order: %.10f" %E1_2nd_corrected)

  #n = 2
  E2_1st_corrected, E2_2nd_corrected = nondeg_PT(2, Nstate, V_0)
  print ("E2 corrected to the 1st order: %.10f" %E2_1st_corrected)
  print ("E2 corrected to the 2nd order: %.10f" %E2_2nd_corrected)
  print ("\n")

Please submit your work by **pasting the link** to this Google Colab notebook to the corresponding assignment on Canvas. To do that, you can click "Share" on the top right of this page, and then **add my email** (ymao2@sdsu.edu) to the top of the pop-up box, and then click the "copy link" button. If you have difficulty doing this, make sure to talk to me sometime next week so that I can help.

This assignment is due at **5:00 PM (PDT) next Friday** (11/04/2022).